In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd

import yfinance as yf

%matplotlib inline
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from scipy.stats import shapiro
from scipy.stats import norm
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings("ignore")

In [13]:
# Carrega a base

df1 = yf.download("^BVSP", "2005-01-01", "2021-12-31")

[*********************100%***********************]  1 of 1 completed


In [14]:
# Construção dos alvos

# Alvo 1 - Retorno
df1["Retorno"] = df1["Adj Close"].pct_change(1)
df1["Alvo1"] = df1["Retorno"].shift(-1)*100

# Alvo 5 - Retorno
df1["Retorno5"] = df1["Adj Close"].pct_change(5)
df1["Alvo5"] = df1["Retorno5"].shift(-5)*100

# Alvo 10 - Retorno
df1["Retorno10"] = df1["Adj Close"].pct_change(10)
df1["Alvo10"] = df1["Retorno10"].shift(-10)*100

# Criação dos alvos categóricos
df1["Alvo1_cat"] = np.where(df1["Alvo1"] > 0 , 1, 0)
df1["Alvo5_cat"] = np.where(df1["Alvo5"] > 0 , 1, 0)
df1["Alvo10_cat"] = np.where(df1["Alvo10"] > 0 , 1, 0)

df1.dropna(inplace = True)

In [15]:
df1.head(20)

,Open,High,Low,Close,Adj Close,Volume,Retorno,Alvo1,Retorno5,Alvo5,Retorno10,Alvo10,Alvo1_cat,Alvo5_cat,Alvo10_cat
Date,,,,,,,,,,,,,,,
2005-01-17,24924.0,25022.0,24515.0,24515.0,24515.0,0,-0.016410,-1.737712,0.009180,-1.293086,-0.046925,-1.492963,0,0,0
2005-01-18,24515.0,24515.0,24019.0,24089.0,24089.0,0,-0.017377,0.755532,-0.011531,1.830711,-0.030546,2.142057,1,1,1
2005-01-19,24091.0,24465.0,24091.0,24271.0,24271.0,0,0.007555,-2.723415,-0.009751,-0.992955,-0.017050,2.484446,0,0,1
2005-01-20,24271.0,24271.0,23534.0,23610.0,23610.0,0,-0.027234,0.880983,-0.048214,1.516307,-0.031067,8.987717,1,1,1
2005-01-21,23618.0,24006.0,23609.0,23818.0,23818.0,0,0.008810,1.595432,-0.044375,2.237803,-0.037540,10.479469,1,1,1
2005-01-24,23819.0,24302.0,23813.0,24198.0,24198.0,0,0.015954,1.372014,-0.012931,-0.202496,-0.003870,9.166047,1,0,1
2005-01-26,24207.0,24555.0,24207.0,24530.0,24530.0,0,0.013720,-2.038320,0.018307,0.305748,0.006565,8.728088,0,1,1
2005-01-27,24529.0,24529.0,23869.0,24030.0,24030.0,0,-0.020383,-0.258011,-0.009930,3.512276,-0.019584,10.411985,0,1,1
2005-01-28,24006.0,24056.0,23595.0,23968.0,23968.0,0,-0.002580,1.597964,0.015163,7.359813,-0.033782,11.023031,1,1,1


In [16]:
# Analisando as distribuições
fig = go.Figure(data=[go.Histogram(x = df1["Alvo1"])])

fig.update_layout(height = 400, width = 600
                  , title_text = "Distribuição dos alvos"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "1 dia"
                  , yaxis_title = "Frequency"
                  , font = dict(size = 15, color = "Black")
                 )

fig.update_layout(hovermode = "x")
fig.show()

In [17]:
df1["Alvo1"].describe()

count    4181.000000
mean        0.050931
std         1.753334
min       -14.779679
25%        -0.852181
50%         0.074476
75%         0.988602
max        14.655996
Name: Alvo1, dtype: float64

In [18]:
# Analisando as distribuições
fig = go.Figure(data=[go.Histogram(x = df1["Alvo5"])])

fig.update_layout(height = 400, width = 600
                  , title_text = "Distribuição dos alvos"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "5 dias"
                  , yaxis_title = "Frequency"
                  , font = dict(size = 15, color = "Black")
                 )

fig.update_layout(hovermode = "x")
fig.show()

In [19]:
# Analisando as distribuições
fig = go.Figure(data=[go.Histogram(x = df1["Alvo10"])])

fig.update_layout(height = 400, width = 600
                  , title_text = "Distribuição dos alvos"
                  , font_color = "blue"
                  , title_font_color = "black"
                  , xaxis_title = "10 dias"
                  , yaxis_title = "Frequency"
                  , font = dict(size = 15, color = "Black")
                 )

fig.update_layout(hovermode = "x")
fig.show()

In [22]:
# Os alvos tem distribuição normal

stat, p = shapiro(df1["Alvo10"])

# p <= alpha: reject H0, not normal.
# p > alpha: fail to reject H0, normal.
    
alpha = 0.05
if p > alpha:
    msg = 'Gaussian Sample (do not reject H0)'
else:
    msg = 'Non Gaussian Sample(reject H0)'

result_mat = [
    ["Sample size", "T stat", 'p-value', 'Comment'],
    [len(df1["Alvo1"]), np.round(stat, 4), np.round(p, 4), msg]
]
swt_table = ff.create_table(result_mat)
swt_table['data'][0].colorscale=[[0, 'blue'],[1, '#ffffff']]
swt_table['layout']['height'] = 100
swt_table['layout']['width'] = 930
swt_table['layout']['margin']['t'] = 10
swt_table['layout']['margin']['b'] = 10
swt_table

In [23]:
df1["Alvo1"].describe()

count    4181.000000
mean        0.050931
std         1.753334
min       -14.779679
25%        -0.852181
50%         0.074476
75%         0.988602
max        14.655996
Name: Alvo1, dtype: float64

In [24]:
print("Com: ")
print("    50% de probabilidade, o alvo será maior do que " + str(round(np.median(df1["Alvo1"]), 2)) + "%")
print("    25% de probabilidade, o alvo será maior do que " + str(round(np.percentile(df1["Alvo1"], 75), 2)) + "%")
print("     5% de probabilidade, o alvo será maior do que " + str(round(np.percentile(df1["Alvo1"], 95), 2)) + "%")
print("    25% de probabilidade, o alvo será menor do que " + str(round(np.percentile(df1["Alvo1"], 25), 2)) + "%")
print("     5% de probabilidade, o alvo será menor do que " + str(round(np.percentile(df1["Alvo1"], 5), 2)) + "%")


Com: 
    50% de probabilidade, o alvo será maior do que 0.07%
    25% de probabilidade, o alvo será maior do que 0.99%
     5% de probabilidade, o alvo será maior do que 2.55%
    25% de probabilidade, o alvo será menor do que -0.85%
     5% de probabilidade, o alvo será menor do que -2.65%


1 - para compra; apenas acima do percentil 75%
0 - não faz nada;25%-75%
-1- para venda; apenas quando abaixo do percentil 25%